In [1]:
import scipy.misc
import os
import tensorflow as tf
import scipy.misc
import model
import cv2
from subprocess import call
import random
import scipy
from tensorflow.core.protobuf import saver_pb2

#### Loading DataSet(train:test::70:30)

In [36]:
xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


train_xs = xs[:int(len(xs) * 0.7)]
train_ys = ys[:int(len(xs) * 0.7)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66, 200]) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66, 200]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out


In [37]:
print("num_val_images",num_val_images)
print("num_train_images",num_train_images)

num_val_images 13621
num_train_images 31784


#### Defining Model(Tanh Activation)

In [3]:
def weight_variable(shape):
    
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, stride):
    return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.multiply(tf.tanh(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) #scale the atan output


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


#### Training model
* optimizer - Adam(1e-3)
* dropout  - 0.5

In [52]:
sess.close()

In [53]:
LOGDIR = "tanh_1"


sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V2)

# op to write logs to Tensorboard
logs_path = 'logs_tanh_1'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(num_images/batch_size)):
        xs, ys = LoadTrainBatch(batch_size)
        train_step.run(feed_dict={x: xs, y_: ys,keep_prob: 0.5})
        if i % 10 == 0:
            xs, ys = LoadValBatch(batch_size)
            loss_value = loss.eval(feed_dict={x:xs, y_: ys, keep_prob: 0.5})
            print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

        # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={x:xs, y_: ys, keep_prob: 0.5})
        summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                 os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
    print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")


C:\Users\sandeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\Users\sandeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
C:\Users\sandeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\Users\sandeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Epoch: 0, Step: 0, Loss: 13.4524
Epoch: 0, Step: 10, Loss: 10.7093
Epoch: 0, Step: 20, Loss: 9.32776
Epoch: 0, Step: 30, Loss: 8.21336
Epoch: 0, Step: 40, Loss: 7.46301
Epoch: 0, Step: 50, Loss: 6.44232
Epoch: 0, Step: 60, Loss: 5.95715
Epoch: 0, Step: 70, Loss: 5.61496
Epoch: 0, Step: 80, Loss: 4.98488
Epoch: 0, Step: 90, Loss: 4.32383
Epoch: 0, Step: 100, Loss: 3.94732
Epoch: 0, Step: 110, Loss: 3.59848
Epoch: 0, Step: 120, Loss: 3.35931
Epoch: 0, Step: 130, Loss: 3.4449
Epoch: 0, Step: 140, Loss: 3.77016
Epoch: 0, Step: 150, Loss: 2.98955
Epoch: 0, Step: 160, Loss: 3.73831
Epoch: 0, Step: 170, Loss: 2.65916
Epoch: 0, Step: 180, Loss: 2.90452
Epoch: 0, Step: 190, Loss: 2.53917
Epoch: 0, Step: 200, Loss: 2.08161
Epoch: 0, Step: 210, Loss: 1.97903
Epoch: 0, Step: 220, Loss: 1.92824
Epoch: 0, Step: 230, Loss: 1.95143
Epoch: 0, Step: 240, Loss: 2.78047
Epoch: 0, Step: 250, Loss: 1.85899
Epoch: 0, Step: 260, Loss: 4.51709
Epoch: 0, Step: 270, Loss: 2.11252
Epoch: 0, Step: 280, Loss: 2.684

Epoch: 4, Step: 810, Loss: 0.0423367
Epoch: 4, Step: 820, Loss: 0.0417362
Epoch: 4, Step: 830, Loss: 0.0410294
Epoch: 4, Step: 840, Loss: 0.0406705
Epoch: 4, Step: 850, Loss: 0.0410172
Model saved in file: tanh_1\model.ckpt
Epoch: 5, Step: 500, Loss: 0.0397837
Epoch: 5, Step: 510, Loss: 0.0416269
Epoch: 5, Step: 520, Loss: 0.0404112
Epoch: 5, Step: 530, Loss: 0.0406713
Epoch: 5, Step: 540, Loss: 0.0403517
Epoch: 5, Step: 550, Loss: 0.040093
Epoch: 5, Step: 560, Loss: 0.0382624
Epoch: 5, Step: 570, Loss: 0.0384359
Epoch: 5, Step: 580, Loss: 0.0469872
Epoch: 5, Step: 590, Loss: 0.0932149
Epoch: 5, Step: 600, Loss: 2.80628
Epoch: 5, Step: 610, Loss: 2.78725
Epoch: 5, Step: 620, Loss: 0.0365284
Epoch: 5, Step: 630, Loss: 0.0344687
Epoch: 5, Step: 640, Loss: 0.0945941
Epoch: 5, Step: 650, Loss: 0.100941
Epoch: 5, Step: 660, Loss: 0.0844027
Epoch: 5, Step: 670, Loss: 0.0825669
Epoch: 5, Step: 680, Loss: 0.140961
Epoch: 5, Step: 690, Loss: 0.0345491
Epoch: 5, Step: 700, Loss: 0.057674
Epoch: 

Epoch: 9, Step: 1180, Loss: 0.0744728
Epoch: 9, Step: 1190, Loss: 0.109032
Epoch: 9, Step: 1200, Loss: 0.00611997
Epoch: 9, Step: 1210, Loss: 0.20478
Epoch: 9, Step: 1220, Loss: 0.326913
Epoch: 9, Step: 1230, Loss: 0.481876
Epoch: 9, Step: 1240, Loss: 1.25848
Epoch: 9, Step: 1250, Loss: 0.331862
Epoch: 9, Step: 1260, Loss: 0.00472797
Epoch: 9, Step: 1270, Loss: 0.0127289
Epoch: 9, Step: 1280, Loss: 0.0295959
Epoch: 9, Step: 1290, Loss: 0.041025
Epoch: 9, Step: 1300, Loss: 0.2826
Epoch: 9, Step: 1310, Loss: 0.60489
Epoch: 9, Step: 1320, Loss: 0.267701
Epoch: 9, Step: 1330, Loss: 0.0890684
Epoch: 9, Step: 1340, Loss: 0.0425066
Epoch: 9, Step: 1350, Loss: 0.0160519
Model saved in file: tanh_1\model.ckpt
Epoch: 10, Step: 1000, Loss: 0.120085
Epoch: 10, Step: 1010, Loss: 0.846398
Epoch: 10, Step: 1020, Loss: 1.49638
Epoch: 10, Step: 1030, Loss: 0.230918
Epoch: 10, Step: 1040, Loss: 0.117964
Epoch: 10, Step: 1050, Loss: 0.00338801
Epoch: 10, Step: 1060, Loss: 0.00920668
Epoch: 10, Step: 1070

Epoch: 14, Step: 1450, Loss: 5.07234
Epoch: 14, Step: 1460, Loss: 0.474277
Epoch: 14, Step: 1470, Loss: 0.00282684
Epoch: 14, Step: 1480, Loss: 0.0189844
Epoch: 14, Step: 1490, Loss: 0.0569941
Epoch: 14, Step: 1500, Loss: 0.0662802
Epoch: 14, Step: 1510, Loss: 0.00867387
Epoch: 14, Step: 1520, Loss: 0.101283
Epoch: 14, Step: 1530, Loss: 0.0788526
Epoch: 14, Step: 1540, Loss: 0.00167967
Epoch: 14, Step: 1550, Loss: 0.0531138
Epoch: 14, Step: 1560, Loss: 0.0116652
Epoch: 14, Step: 1570, Loss: 0.0150742
Epoch: 14, Step: 1580, Loss: 0.0088674
Epoch: 14, Step: 1590, Loss: 0.158309
Epoch: 14, Step: 1600, Loss: 0.0377088
Epoch: 14, Step: 1610, Loss: 0.129969
Epoch: 14, Step: 1620, Loss: 0.0722601
Epoch: 14, Step: 1630, Loss: 0.0338466
Epoch: 14, Step: 1640, Loss: 0.0208561
Epoch: 14, Step: 1650, Loss: 0.0210524
Epoch: 14, Step: 1660, Loss: 0.0214869
Epoch: 14, Step: 1670, Loss: 0.0229502
Epoch: 14, Step: 1680, Loss: 0.0228751
Epoch: 14, Step: 1690, Loss: 0.0217156
Epoch: 14, Step: 1700, Loss:

Epoch: 18, Step: 2090, Loss: 0.896444
Epoch: 18, Step: 2100, Loss: 0.166556
Epoch: 18, Step: 2110, Loss: 0.00748171
Epoch: 18, Step: 2120, Loss: 0.016693
Epoch: 18, Step: 2130, Loss: 0.0252547
Epoch: 18, Step: 2140, Loss: 0.0348078
Epoch: 18, Step: 2150, Loss: 0.49622
Epoch: 18, Step: 2160, Loss: 0.614211
Epoch: 18, Step: 2170, Loss: 0.206144
Epoch: 18, Step: 2180, Loss: 0.0454703
Epoch: 18, Step: 2190, Loss: 0.0349584
Epoch: 18, Step: 2200, Loss: 0.0187935
Epoch: 18, Step: 2210, Loss: 0.165909
Epoch: 18, Step: 2220, Loss: 1.23486
Epoch: 18, Step: 2230, Loss: 1.10778
Epoch: 18, Step: 2240, Loss: 0.049636
Epoch: 18, Step: 2250, Loss: 0.0864103
Model saved in file: tanh_1\model.ckpt
Epoch: 19, Step: 1900, Loss: 0.00262252
Epoch: 19, Step: 1910, Loss: 0.00717303
Epoch: 19, Step: 1920, Loss: 0.00867721
Epoch: 19, Step: 1930, Loss: 0.0291196
Epoch: 19, Step: 1940, Loss: 0.0700671
Epoch: 19, Step: 1950, Loss: 0.0141229
Epoch: 19, Step: 1960, Loss: 0.0165611
Epoch: 19, Step: 1970, Loss: 0.116

Epoch: 23, Step: 2350, Loss: 0.0748411
Epoch: 23, Step: 2360, Loss: 0.0156513
Epoch: 23, Step: 2370, Loss: 0.13277
Epoch: 23, Step: 2380, Loss: 0.0232131
Epoch: 23, Step: 2390, Loss: 0.00790843
Epoch: 23, Step: 2400, Loss: 0.0498697
Epoch: 23, Step: 2410, Loss: 0.0106582
Epoch: 23, Step: 2420, Loss: 0.0177454
Epoch: 23, Step: 2430, Loss: 0.0556184
Epoch: 23, Step: 2440, Loss: 0.10761
Epoch: 23, Step: 2450, Loss: 0.0767338
Epoch: 23, Step: 2460, Loss: 0.137441
Epoch: 23, Step: 2470, Loss: 0.0340611
Epoch: 23, Step: 2480, Loss: 0.0315299
Epoch: 23, Step: 2490, Loss: 0.017361
Epoch: 23, Step: 2500, Loss: 0.0154228
Epoch: 23, Step: 2510, Loss: 0.0176075
Epoch: 23, Step: 2520, Loss: 0.018724
Epoch: 23, Step: 2530, Loss: 0.0190917
Epoch: 23, Step: 2540, Loss: 0.0186788
Epoch: 23, Step: 2550, Loss: 0.031327
Epoch: 23, Step: 2560, Loss: 0.0247821
Epoch: 23, Step: 2570, Loss: 0.032907
Epoch: 23, Step: 2580, Loss: 0.00420741
Epoch: 23, Step: 2590, Loss: 0.031919
Epoch: 23, Step: 2600, Loss: 0.09

Epoch: 27, Step: 2990, Loss: 0.135643
Epoch: 27, Step: 3000, Loss: 0.596714
Epoch: 27, Step: 3010, Loss: 0.439579
Epoch: 27, Step: 3020, Loss: 0.127597
Epoch: 27, Step: 3030, Loss: 0.0443804
Epoch: 27, Step: 3040, Loss: 0.0173325
Epoch: 27, Step: 3050, Loss: 0.0648588
Epoch: 27, Step: 3060, Loss: 0.515482
Epoch: 27, Step: 3070, Loss: 1.42931
Epoch: 27, Step: 3080, Loss: 0.543223
Epoch: 27, Step: 3090, Loss: 0.114722
Epoch: 27, Step: 3100, Loss: 0.0165736
Epoch: 27, Step: 3110, Loss: 0.00561435
Epoch: 27, Step: 3120, Loss: 0.000989006
Epoch: 27, Step: 3130, Loss: 0.0101745
Epoch: 27, Step: 3140, Loss: 0.0293084
Epoch: 27, Step: 3150, Loss: 0.0483373
Model saved in file: tanh_1\model.ckpt
Epoch: 28, Step: 2800, Loss: 0.00261217
Epoch: 28, Step: 2810, Loss: 0.0858872
Epoch: 28, Step: 2820, Loss: 0.208007
Epoch: 28, Step: 2830, Loss: 0.279733
Epoch: 28, Step: 2840, Loss: 0.203321
Epoch: 28, Step: 2850, Loss: 0.0100287
Epoch: 28, Step: 2860, Loss: 0.00805446
Epoch: 28, Step: 2870, Loss: 0.0

In [56]:

#sess = tf.InteractiveSession()
#saver = tf.train.Saver()
#saver.restore(sess, "save_copy/model.ckpt")

#img = cv2.imread('steering_wheel_image.jpg',0)
#rows,cols = img.shape

#smoothed_angle = 0
Predicted_1 = []
#cap = cv2.VideoCapture(0)
for i in range(len(val_xs)):
    full_image = scipy.misc.imread(val_xs[i], mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    degrees = sess.run(y,feed_dict={x: [image], keep_prob: 1.0})[0][0] * 180 / scipy.pi
    Predicted_1.append(degrees)
   #call("clear")
    #print("Predicted steering angle: " + str(degrees) + " degrees")
    #cv2.imshow('frame', full_image)
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    #smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    #M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    #dst = cv2.warpAffine(img,M,(cols,rows))
    #cv2.imshow("steering wheel", dst)

#cap.release()
#cv2.destroyAllWindows()


C:\Users\sandeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
C:\Users\sandeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


In [58]:
Predicted_1[0:5]

[1.0671196208971139,
 1.0671196208971139,
 1.0671196208971139,
 1.0671196208971139,
 1.0671196208971139]

* Getting constant angle with learning rate of 1e-3, so we will change the learning rate

#### Training model
* optimizer - Adam(1e-4)
* dropout  - 0.5

In [7]:
LOGDIR = "tanh"


sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V2)

# op to write logs to Tensorboard
logs_path = 'logs_tanh'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(num_images/batch_size)):
        xs, ys = LoadTrainBatch(batch_size)
        train_step.run(feed_dict={x: xs, y_: ys,keep_prob: 0.5})
        if i % 10 == 0:
            xs, ys = LoadValBatch(batch_size)
            loss_value = loss.eval(feed_dict={x:xs, y_: ys, keep_prob: 0.5})
            print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

        # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={x:xs, y_: ys, keep_prob: 0.5})
        summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                 os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
    print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")


C:\Users\sandeep\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
C:\Users\sandeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\Users\sandeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
C:\Users\sandeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Us

Epoch: 0, Step: 0, Loss: 8.67142
Epoch: 0, Step: 10, Loss: 8.27446
Epoch: 0, Step: 20, Loss: 7.83546
Epoch: 0, Step: 30, Loss: 7.10138
Epoch: 0, Step: 40, Loss: 6.7688
Epoch: 0, Step: 50, Loss: 6.64455
Epoch: 0, Step: 60, Loss: 6.5605
Epoch: 0, Step: 70, Loss: 6.29671
Epoch: 0, Step: 80, Loss: 6.075
Epoch: 0, Step: 90, Loss: 5.97259
Epoch: 0, Step: 100, Loss: 5.83923
Epoch: 0, Step: 110, Loss: 5.8881
Epoch: 0, Step: 120, Loss: 6.15351
Epoch: 0, Step: 130, Loss: 6.62697
Epoch: 0, Step: 140, Loss: 5.94324
Epoch: 0, Step: 150, Loss: 6.44653
Epoch: 0, Step: 160, Loss: 5.85581
Epoch: 0, Step: 170, Loss: 6.23931
Epoch: 0, Step: 180, Loss: 5.93702
Epoch: 0, Step: 190, Loss: 5.51271
Epoch: 0, Step: 200, Loss: 5.44926
Epoch: 0, Step: 210, Loss: 5.41906
Epoch: 0, Step: 220, Loss: 5.54314
Epoch: 0, Step: 230, Loss: 6.49994
Epoch: 0, Step: 240, Loss: 5.5294
Epoch: 0, Step: 250, Loss: 7.15849
Epoch: 0, Step: 260, Loss: 5.83518
Epoch: 0, Step: 270, Loss: 6.53537
Epoch: 0, Step: 280, Loss: 5.30299
Ep

Epoch: 5, Step: 500, Loss: 2.05754
Epoch: 5, Step: 510, Loss: 2.04851
Epoch: 5, Step: 520, Loss: 2.04122
Epoch: 5, Step: 530, Loss: 2.03266
Epoch: 5, Step: 540, Loss: 2.02338
Epoch: 5, Step: 550, Loss: 2.01279
Epoch: 5, Step: 560, Loss: 2.00808
Epoch: 5, Step: 570, Loss: 2.01438
Epoch: 5, Step: 580, Loss: 2.05652
Epoch: 5, Step: 590, Loss: 4.8439
Epoch: 5, Step: 600, Loss: 4.81554
Epoch: 5, Step: 610, Loss: 1.96796
Epoch: 5, Step: 620, Loss: 1.95306
Epoch: 5, Step: 630, Loss: 1.98486
Epoch: 5, Step: 640, Loss: 1.98046
Epoch: 5, Step: 650, Loss: 1.96108
Epoch: 5, Step: 660, Loss: 1.9823
Epoch: 5, Step: 670, Loss: 2.03999
Epoch: 5, Step: 680, Loss: 1.90208
Epoch: 5, Step: 690, Loss: 1.92515
Epoch: 5, Step: 700, Loss: 1.91005
Epoch: 5, Step: 710, Loss: 1.89858
Epoch: 5, Step: 720, Loss: 1.88837
Epoch: 5, Step: 730, Loss: 1.93687
Epoch: 5, Step: 740, Loss: 1.93025
Epoch: 5, Step: 750, Loss: 1.95225
Epoch: 5, Step: 760, Loss: 1.97975
Epoch: 5, Step: 770, Loss: 1.84395
Epoch: 5, Step: 780, L

Epoch: 9, Step: 1340, Loss: 0.785374
Epoch: 9, Step: 1350, Loss: 0.898736
Model saved in file: tanh\model.ckpt
Epoch: 10, Step: 1000, Loss: 1.64252
Epoch: 10, Step: 1010, Loss: 2.3225
Epoch: 10, Step: 1020, Loss: 1.0049
Epoch: 10, Step: 1030, Loss: 0.843637
Epoch: 10, Step: 1040, Loss: 0.747605
Epoch: 10, Step: 1050, Loss: 0.753622
Epoch: 10, Step: 1060, Loss: 0.744831
Epoch: 10, Step: 1070, Loss: 0.750628
Epoch: 10, Step: 1080, Loss: 0.786754
Epoch: 10, Step: 1090, Loss: 0.782588
Epoch: 10, Step: 1100, Loss: 0.738328
Epoch: 10, Step: 1110, Loss: 0.830719
Epoch: 10, Step: 1120, Loss: 1.10798
Epoch: 10, Step: 1130, Loss: 0.799526
Epoch: 10, Step: 1140, Loss: 0.870581
Epoch: 10, Step: 1150, Loss: 0.713073
Epoch: 10, Step: 1160, Loss: 0.709611
Epoch: 10, Step: 1170, Loss: 0.70677
Epoch: 10, Step: 1180, Loss: 0.704742
Epoch: 10, Step: 1190, Loss: 0.713011
Epoch: 10, Step: 1200, Loss: 5.24789
Epoch: 10, Step: 1210, Loss: 1.69152
Epoch: 10, Step: 1220, Loss: 0.696989
Epoch: 10, Step: 1230, L

Epoch: 14, Step: 1670, Loss: 0.310643
Epoch: 14, Step: 1680, Loss: 0.312947
Epoch: 14, Step: 1690, Loss: 0.307654
Epoch: 14, Step: 1700, Loss: 0.327114
Epoch: 14, Step: 1710, Loss: 0.321166
Epoch: 14, Step: 1720, Loss: 0.300839
Epoch: 14, Step: 1730, Loss: 0.302634
Epoch: 14, Step: 1740, Loss: 0.35001
Epoch: 14, Step: 1750, Loss: 0.31689
Epoch: 14, Step: 1760, Loss: 0.644603
Epoch: 14, Step: 1770, Loss: 0.301236
Epoch: 14, Step: 1780, Loss: 0.350879
Epoch: 14, Step: 1790, Loss: 0.356784
Epoch: 14, Step: 1800, Loss: 0.639564
Epoch: 14, Step: 1810, Loss: 0.324724
Epoch: 14, Step: 1820, Loss: 0.426039
Epoch: 14, Step: 1830, Loss: 0.60927
Epoch: 14, Step: 1840, Loss: 0.514432
Epoch: 14, Step: 1850, Loss: 0.314235
Model saved in file: tanh\model.ckpt
Epoch: 15, Step: 1500, Loss: 0.306621
Epoch: 15, Step: 1510, Loss: 0.281584
Epoch: 15, Step: 1520, Loss: 0.308304
Epoch: 15, Step: 1530, Loss: 0.498167
Epoch: 15, Step: 1540, Loss: 1.07197
Epoch: 15, Step: 1550, Loss: 0.563155
Epoch: 15, Step: 

Epoch: 19, Step: 1990, Loss: 0.164059
Epoch: 19, Step: 2000, Loss: 0.157414
Epoch: 19, Step: 2010, Loss: 0.157136
Epoch: 19, Step: 2020, Loss: 0.157277
Epoch: 19, Step: 2030, Loss: 0.155997
Epoch: 19, Step: 2040, Loss: 0.518425
Epoch: 19, Step: 2050, Loss: 5.14791
Epoch: 19, Step: 2060, Loss: 0.221449
Epoch: 19, Step: 2070, Loss: 0.163682
Epoch: 19, Step: 2080, Loss: 0.152204
Epoch: 19, Step: 2090, Loss: 0.155972
Epoch: 19, Step: 2100, Loss: 0.155929
Epoch: 19, Step: 2110, Loss: 0.166553
Epoch: 19, Step: 2120, Loss: 0.157458
Epoch: 19, Step: 2130, Loss: 0.154918
Epoch: 19, Step: 2140, Loss: 0.154941
Epoch: 19, Step: 2150, Loss: 0.154246
Epoch: 19, Step: 2160, Loss: 0.156384
Epoch: 19, Step: 2170, Loss: 0.168857
Epoch: 19, Step: 2180, Loss: 0.184945
Epoch: 19, Step: 2190, Loss: 0.161833
Epoch: 19, Step: 2200, Loss: 0.153881
Epoch: 19, Step: 2210, Loss: 0.150509
Epoch: 19, Step: 2220, Loss: 0.19826
Epoch: 19, Step: 2230, Loss: 0.151035
Epoch: 19, Step: 2240, Loss: 0.149377
Epoch: 19, Ste

Epoch: 23, Step: 2680, Loss: 0.375966
Epoch: 23, Step: 2690, Loss: 0.283848
Epoch: 23, Step: 2700, Loss: 0.168773
Epoch: 23, Step: 2710, Loss: 0.115059
Epoch: 23, Step: 2720, Loss: 0.123097
Epoch: 23, Step: 2730, Loss: 0.119843
Epoch: 23, Step: 2740, Loss: 0.474335
Epoch: 23, Step: 2750, Loss: 0.334446
Model saved in file: tanh\model.ckpt
Epoch: 24, Step: 2400, Loss: 0.706015
Epoch: 24, Step: 2410, Loss: 0.883716
Epoch: 24, Step: 2420, Loss: 0.168312
Epoch: 24, Step: 2430, Loss: 0.448542
Epoch: 24, Step: 2440, Loss: 0.223065
Epoch: 24, Step: 2450, Loss: 0.108986
Epoch: 24, Step: 2460, Loss: 0.117585
Epoch: 24, Step: 2470, Loss: 0.213357
Epoch: 24, Step: 2480, Loss: 0.340996
Epoch: 24, Step: 2490, Loss: 0.29902
Epoch: 24, Step: 2500, Loss: 0.723441
Epoch: 24, Step: 2510, Loss: 1.93439
Epoch: 24, Step: 2520, Loss: 0.326169
Epoch: 24, Step: 2530, Loss: 0.367988
Epoch: 24, Step: 2540, Loss: 0.141236
Epoch: 24, Step: 2550, Loss: 0.127911
Epoch: 24, Step: 2560, Loss: 0.763826
Epoch: 24, Step

Epoch: 28, Step: 2990, Loss: 0.0890726
Epoch: 28, Step: 3000, Loss: 0.0931663
Epoch: 28, Step: 3010, Loss: 0.100707
Epoch: 28, Step: 3020, Loss: 0.137797
Epoch: 28, Step: 3030, Loss: 0.129594
Epoch: 28, Step: 3040, Loss: 0.0892257
Epoch: 28, Step: 3050, Loss: 0.0860704
Epoch: 28, Step: 3060, Loss: 0.0904601
Epoch: 28, Step: 3070, Loss: 0.0997275
Epoch: 28, Step: 3080, Loss: 0.0917657
Epoch: 28, Step: 3090, Loss: 0.0982597
Epoch: 28, Step: 3100, Loss: 0.0973418
Epoch: 28, Step: 3110, Loss: 0.0888692
Epoch: 28, Step: 3120, Loss: 0.10876
Epoch: 28, Step: 3130, Loss: 0.130965
Epoch: 28, Step: 3140, Loss: 0.157345
Epoch: 28, Step: 3150, Loss: 0.119979
Epoch: 28, Step: 3160, Loss: 0.111931
Epoch: 28, Step: 3170, Loss: 0.113716
Epoch: 28, Step: 3180, Loss: 0.112554
Epoch: 28, Step: 3190, Loss: 0.176943
Epoch: 28, Step: 3200, Loss: 0.103552
Epoch: 28, Step: 3210, Loss: 5.08565
Epoch: 28, Step: 3220, Loss: 0.733533
Epoch: 28, Step: 3230, Loss: 0.087768
Epoch: 28, Step: 3240, Loss: 0.0998776
Epo

#### Prediction on test dataset

In [10]:
#sess = tf.InteractiveSession()
#saver = tf.train.Saver()
#saver.restore(sess, "save_copy/model.ckpt")

#img = cv2.imread('steering_wheel_image.jpg',0)
#rows,cols = img.shape

#smoothed_angle = 0
Predicted = []
#cap = cv2.VideoCapture(0)
for i in range(len(val_xs)):
    full_image = scipy.misc.imread(val_xs[i], mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    degrees = sess.run(y,feed_dict={x: [image], keep_prob: 1.0})[0][0] * 180 / scipy.pi
    Predicted.append(degrees)
    #call("clear")
    #print("Predicted steering angle: " + str(degrees) + " degrees")
    #cv2.imshow('frame', full_image)
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    #smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    #M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    #dst = cv2.warpAffine(img,M,(cols,rows))
    #cv2.imshow("steering wheel", dst)

#cap.release()
#cv2.destroyAllWindows()


C:\Users\sandeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\Users\sandeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [11]:
len(Predicted)

13621

In [12]:
import pandas as pd

In [14]:
df_degrees = pd.DataFrame(Predicted, columns = ["predicted_angle"])

In [15]:
df_degrees.head()

,predicted_angle
0,-23.057569
1,-19.903451
2,-17.324210
3,-19.595773
4,-19.223354


In [16]:
df_degrees.to_csv("degrees_pred.csv")

In [17]:
len(val_ys)

13621

In [18]:
true_degrees = [val_ys[i]* 180 / scipy.pi for i in range(0,len(val_ys))]

In [20]:
df_degrees["true"] = true_degrees

In [21]:
df_degrees.head()

,predicted_angle,true
0,-23.057569,-28.34
1,-19.903451,-28.84
2,-17.324210,-29.75
3,-19.595773,-31.06
4,-19.223354,-32.27


In [24]:
df_degrees.to_csv("pred_true.csv")

#### Prediction on whole dataset(train+test)

In [38]:
import tensorflow as tf
import scipy.misc
import model
import cv2
from subprocess import call

#sess = tf.InteractiveSession()
#saver = tf.train.Saver()
#saver.restore(sess, "save_copy/model.ckpt")

#img = cv2.imread('steering_wheel_image.jpg',0)
#rows,cols = img.shape

#smoothed_angle = 0
Predicted_all = []
#cap = cv2.VideoCapture(0)
for i in range(len(xs)):
    full_image = scipy.misc.imread(xs[i], mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    degrees = sess.run(y,feed_dict={x: [image], keep_prob: 1.0})[0][0] * 180 / scipy.pi
    Predicted_all.append(degrees)
    #call("clear")
    #print("Predicted steering angle: " + str(degrees) + " degrees")
    #cv2.imshow('frame', full_image)
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    #smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    #M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    #dst = cv2.warpAffine(img,M,(cols,rows))
    #cv2.imshow("steering wheel", dst)

#cap.release()
#cv2.destroyAllWindows()


C:\Users\sandeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\Users\sandeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [39]:
len(Predicted_all)

45406

In [41]:
df_degrees_all = pd.DataFrame(Predicted_all, columns = ["predicted_angle"])

In [43]:
df_degrees_all.to_csv("degrees_pred_all.csv")

In [44]:
true_degrees_all = [ys[i]* 180 / scipy.pi for i in range(0,len(ys))]

In [45]:
df_degrees_all["true"] = true_degrees_all

#### Visualizing the output using CV2

In [64]:
import tensorflow as tf
import scipy.misc
import model
import cv2
from subprocess import call
import math

#sess = tf.InteractiveSession()
#saver = tf.train.Saver()
#saver.restore(sess, "save_new/model.ckpt")

img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0


#read data.txt
#get number of images
#num_images = len(xs)


i = 0
#print("Starting frameofvideo:" +str(i))

while(cv2.waitKey(10) != ord('q')):
    full_image = scipy.misc.imread("driving_dataset/" + str(i) + ".jpg", mode="RGB")
    degrees = Predicted_all[i]
    #call("clear")
    #print("Predicted Steering angle: " + str(degrees))
    #print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1
cv2.destroyAllWindows()


C:\Users\sandeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
